In [100]:
import pandas as pd
import numpy as np
from pathlib import Path

In [53]:
df = pd.read_excel('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_July_31_2024.xls')

In [54]:
df

,Unnamed: 0,"Parag Parikh Flexi Cap Fund (An open-ended dynamic equity scheme investing across large cap, mid-cap, small-cap stocks)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Monthly Portfolio Statement as on July 31, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Name of the Instrument,ISIN,Industry,Quantity,Market/Fair Value\n (Rs. in Lakhs),% to Net\n Assets,YTM~,YTC^,NaN
3,NaN,Equity & Equity related,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,(a) Listed / awaiting listing on Stock Exchanges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
332,NaN,Tier I Benchmark’s Risk-o-meter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
equity_domestic = ['Equity & Equity related']
arbitrage = ['Arbitrage','Arbitrage & Special Situations']
unlisted = ['(b) Unlisted']
equity_foreign = ['Equity & Equity related Foreign Investments']
derivatives = ['Derivatives']
money_market = ['Money Market Instruments']
options = ['Index / Stock Options']
futures = ['Index / Stock Futures']
end_note = ['Notes:']


In [68]:
def extract_clean_assets_from_fund_single_file(fund_name: str, file_path: Path, table_extract_combos:list):
    df = pd.read_excel(file_path)
    statement_period = extract_statement_period(df)
    df_dict_raw = extract_all_raw_tables(df, table_extract_combos)
    df_list_clean = clean_all_raw_tables(df_dict_raw)
    combined_df = pd.concat(df_list_clean).assign(statement_period=statement_period).reset_index(drop=True)
    return combined_df
    
    

In [154]:
def extract_statement_period(df):
    return df[df.iloc[:,1].astype('str').str.contains('Monthly Portfolio Statement')].iloc[:,1].values[0]

In [72]:
def get_idx_from_instrument_text(df, text, column=1):
    #return df[df.iloc[:, column]==text].index[0]
    return df[df.iloc[:, column].isin(text)].index[0]

In [76]:
df[df.iloc[:, 1].isin(equity_domestic)].index[0]

np.int64(3)

In [62]:
get_idx_from_instrument_text(df,arbitrage)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [15]:
def get_raw_table(df, start_idx_text, end_idx_text):
    start_idx = get_idx_from_instrument_text(df, start_idx_text)
    end_idx = get_idx_from_instrument_text(df, end_idx_text)
    table_df = df.iloc[start_idx:end_idx-1, :]
    return table_df

In [77]:
table_extract_combos

[(['Equity & Equity related'],
  ['Arbitrage', 'Arbitrage & Special Situations']),
 (['Arbitrage', 'Arbitrage & Special Situations'], ['(b) Unlisted']),
 (['Equity & Equity related Foreign Investments'], ['Stock Options']),
 (['Stock Options'], ['Money Market Instruments']),
 (['Money Market Instruments'], ['Index / Stock Futures']),
 (['Index / Stock Futures'], ['Notes:'])]

In [78]:
def extract_all_raw_tables(df, table_extract_combos):
    df_dict = {}
    for start, end in table_extract_combos:
        try:
            df_dict[start[0]] = get_raw_table(df, start, end)
        except Exception as e:
            print(f'Error extracting table for {start} to {end} with error {e}')
    return df_dict

In [108]:
clean_raw_table(extract_all_raw_tables(df, table_extract_combos)['Equity & Equity related'])

Column length mismatch for ['instrument', 'isin', 'industry', 'quantity', 'value', 'percentage_of_net_assets', 'ytm'] ytm added as default


,instrument,isin,industry,quantity,value,percentage_of_net_assets,ytm
5,HDFC Bank Limited,INE040A01034,Banks,37727754.0,609586.19,0.0803,NaN
6,Power Grid Corporation of India Limited,INE752E01010,Power,149680449.0,521187.32,0.0686,NaN
7,Bajaj Holdings & Investment Limited,INE118A01012,Finance,5136174.0,494074.26,0.0650,NaN
8,ITC Limited,INE154A01025,Diversified FMCG,88311955.0,437453.27,0.0576,NaN
9,Coal India Limited,INE522F01014,Consumable Fuels,83575380.0,436430.63,0.0575,NaN
10,Maruti Suzuki India Limited,INE585B01010,Automobiles,3043078.0,399124.02,0.0525,NaN
11,ICICI Bank Limited,INE090A01021,Banks,31693059.0,385038.97,0.0507,NaN
12,HCL Technologies Limited,INE860A01027,IT - Software,18706973.0,307280.74,0.0405,NaN
13,Axis Bank Limited,INE238A01034,Banks,23645558.0,275730.85,0.0363,NaN
14,Kotak Mahindra Bank Limited,INE237A01028,Banks,10565292.0,191009.91,0.0251,NaN


In [132]:
def rename_columns(df, col_list):
    df.columns = col_list
    return df

In [91]:
x = [1,2,3,4,5]
x[:5]

[1, 2, 3, 4, 5]

In [141]:
def clean_raw_table(df):
    if len(df.iloc[:,1:-3].columns) != 7:
        df = df.assign(unknown = np.nan) # if only 6 columns, add a column of NaNs so that indexing below doesnt remove any useful column
    clean_df = (df
                .iloc[:,1:-3]
                .pipe(rename_columns, col_list)
                .query('(~quantity.isnull())|(~ytm.isnull())')
                .assign(
                        quantity = lambda x: x.quantity.astype(float),
                        value = lambda x: x.value.astype(float),
                        percentage_of_net_assets = lambda x: x.percentage_of_net_assets.astype('string').str.replace('$', '').str.replace('%', '').astype(float),
                        ytm = lambda x: x.ytm.astype(float)
                    )
                )
    return clean_df

In [142]:
def clean_all_raw_tables(df_dict):
    clean_df_list = []
    for key,df in df_dict.items():
        try:
            print(key)
            df = clean_raw_table(df).assign(table_name=key)
            clean_df_list.append(df)
        except Exception as e:
            print(e)
            continue
    return clean_df_list

In [113]:
def read_fund_files_path(fund_code: str) -> pd.DataFrame:
    """
    Process all Excel files for a given fund code.
    
    Args:
        fund_code (str): The fund code to process (e.g. 'PPFAS')
        
    Returns:
        pd.DataFrame: Combined processed data from all Excel files
    """
    try:
        # Get data directory path 
        data_dir = Path('data') / fund_code
        
        if not data_dir.exists():
            raise FileNotFoundError(f"Data directory not found: {data_dir}")
            
        # Get all Excel files
        excel_files = list(data_dir.glob('*.xls*'))
        
        if not excel_files:
            print(f"No Excel files found in {data_dir}")
            
        return excel_files
        
    except Exception as e:
        print(f"Error processing files: {e}")
        return []

In [114]:
file_paths = read_fund_files_path('ppfas')

In [82]:
table_extract_combos = [(equity_domestic, arbitrage),(arbitrage, unlisted), (equity_foreign, options), (options,money_market), (money_market, futures),(futures,end_note)]

In [32]:
statement_period = extract_statement_period(df)

In [33]:
statement_period

'Monthly Portfolio Statement as on July 31, 2024'

In [35]:
df

,Unnamed: 0,"Parag Parikh Flexi Cap Fund (An open-ended dynamic equity scheme investing across large cap, mid-cap, small-cap stocks)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Monthly Portfolio Statement as on July 31, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Name of the Instrument,ISIN,Industry,Quantity,Market/Fair Value\n (Rs. in Lakhs),% to Net\n Assets,YTM~,YTC^,NaN
3,NaN,Equity & Equity related,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,(a) Listed / awaiting listing on Stock Exchanges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
332,NaN,Tier I Benchmark’s Risk-o-meter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df_dict = extract_all_raw_tables(df, table_extract_combos)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [43]:
get_raw_table(df, equity_domestic, arbitrage)

,Unnamed: 0,"Parag Parikh Flexi Cap Fund (An open-ended dynamic equity scheme investing across large cap, mid-cap, small-cap stocks)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
3,NaN,Equity & Equity related,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,(a) Listed / awaiting listing on Stock Exchanges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,HDFC Bank Limited,INE040A01034,Banks,37727754,609586.19,0.0803,NaN,NaN,NaN
6,NaN,Power Grid Corporation of India Limited,INE752E01010,Power,149680449,521187.32,0.0686,NaN,NaN,NaN
7,NaN,Bajaj Holdings & Investment Limited,INE118A01012,Finance,5136174,494074.26,0.065,NaN,NaN,NaN
8,NaN,ITC Limited,INE154A01025,Diversified FMCG,88311955,437453.27,0.0576,NaN,NaN,NaN
9,NaN,Coal India Limited,INE522F01014,Consumable Fuels,83575380,436430.63,0.0575,NaN,NaN,NaN
10,NaN,Maruti Suzuki India Limited,INE585B01010,Automobiles,3043078,399124.02,0.0525,NaN,NaN,NaN
11,NaN,ICICI Bank Limited,INE090A01021,Banks,31693059,385038.97,0.0507,NaN,NaN,NaN
12,NaN,HCL Technologies Limited,INE860A01027,IT - Software,18706973,307280.74,0.0405,NaN,NaN,NaN


In [ ]:
df_dict

In [ ]:
df_list = extract_all_raw_tables(df, table_extract_combos)

In [24]:
col_list = ['instrument', 'isin','industry', 'quantity', 'value', 'percentage_of_net_assets','ytm']

In [ ]:
df_list[1]

In [ ]:
df_list[4]

In [ ]:
clean_df_list = clean_all_raw_tables(df_dict)

In [ ]:
combined_df = pd.concat(clean_df_list).assign(statement_period=statement_period).reset_index(drop=True)

In [ ]:
combined_df

In [ ]:
combined_df.groupby('table_name').percentage_of_net_assets.sum()

In [ ]:
get_idx_from_instrument_text(df, derivatives)

In [ ]:
get_raw_table(df, equity_domestic,arbitrage)

In [ ]:
get_idx_from_instrument_text(df,equity_domestic)

In [ ]:
df[df.iloc[:, 1]==equity_domestic].index[0]

In [ ]:
df[df.astype(str).apply(lambda x: x.str.contains('Monthly Portfolio Statement')).any(axis=1)].dropna(how='all', axis=1).values

In [26]:
file_paths

[PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_October_31_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_September_30_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_July_31_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_August_31_2024.xls')]

In [147]:
comb_df_list = []
for f in file_paths:
    try:
        df = extract_clean_assets_from_fund_single_file('ppfas', f, table_extract_combos)
        comb_df_list.append(df)
        print(f'{f}- {df.percentage_of_net_assets.sum()}')
    except Exception as e:
        print(f'Error processing file {f} with error {e}')

Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures
could not convert string to float: '$0.00%'
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_October_31_2024.xls- 1.0002
Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_September_30_2024.xls- 0.9937000000000001
Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_July_31_2024.xls- 0.9919000000000001
Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_August_31_2024.xls- 0.9962000000000001


In [148]:
combined_all_df = pd.concat(comb_df_list)

In [151]:
combined_all_df

,instrument,isin,industry,quantity,value,percentage_of_net_assets,ytm,table_name,statement_period
0,HDFC Bank Limited,INE040A01034,Banks,39684109.0,688797.08,0.0841,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
1,Power Grid Corporation of India Limited,INE752E01010,Power,180756561.0,579867.05,0.0708,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
2,Bajaj Holdings & Investment Limited,INE118A01012,Finance,5328201.0,546113.96,0.0667,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
3,Coal India Limited,INE522F01014,Consumable Fuels,117601373.0,531617.01,0.0649,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
4,ITC Limited,INE154A01025,Diversified FMCG,94519320.0,462010.44,0.0564,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
...,...,...,...,...,...,...,...,...,...
102,Jindal Steel & Power Limited September 2024 Fu...,NaN,Short,-8750.0,-85.27,0.0000,NaN,Index / Stock Futures,"Monthly Portfolio Statement as on August 31, 2024"
103,Bajaj Finserv Limited September 2024 Future,NaN,Short,-4500.0,-80.63,0.0000,NaN,Index / Stock Futures,"Monthly Portfolio Statement as on August 31, 2024"
104,Larsen & Toubro Limited September 2024 Future,NaN,Short,-1200.0,-44.74,0.0000,NaN,Index / Stock Futures,"Monthly Portfolio Statement as on August 31, 2024"
105,Tata Consultancy Services Limited September 20...,NaN,Short,-700.0,-32.06,0.0000,NaN,Index / Stock Futures,"Monthly Portfolio Statement as on August 31, 2024"


In [152]:
combined_all_df.groupby('statement_period').percentage_of_net_assets.sum()

statement_period
Monthly Portfolio Statement as on August 31, 2024       0.9962
Monthly Portfolio Statement as on July 31, 2024         0.9919
Monthly Portfolio Statement as on October 31, 2024      1.0002
Monthly Portfolio Statement as on September 30, 2024    0.9937
Name: percentage_of_net_assets, dtype: float64

In [143]:
check_df = extract_clean_assets_from_fund_single_file('ppfas', file_paths[2], table_extract_combos)

Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures


In [145]:
check_df.groupby('table_name').percentage_of_net_assets.sum()

table_name
Arbitrage                                      0.0072
Equity & Equity related                        0.7104
Equity & Equity related Foreign Investments    0.1284
Index / Stock Futures                         -0.0073
Index / Stock Options                          0.0002
Money Market Instruments                       0.1530
Name: percentage_of_net_assets, dtype: float64

In [144]:
check_df.percentage_of_net_assets.sum()

np.float64(0.9919000000000001)